AI Projects: Kohonen Self Organising Map From Scratch
-----

Kohonen's self-organizing map (SOM) is a type of artificial neural network that can be used for data visualization and unsupervised learning. The SOM is trained to create a low-dimensional representation of high-dimensional data, such as images, sounds, or text. The map consists of a grid of neurons, where each neuron is associated with a weight vector that represents a point in the input space. During training, the SOM adjusts its neurons' weights to match the input data. The result is a map that preserves the topological relationships of the input data, making it easier to visualize and analyze. The SOM can be used for tasks such as clustering, visualization, and feature extraction.

-----
Lets start with importing libraries

In [2]:
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Qt5Agg")
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from PIL import Image


defining some variables for our input data

In [3]:
buyukluk=150 #size of sata
toplam= buyukluk*4 # total =4 * size
egitim_kumesi_noktalar=int(toplam*0.9) #defines the training size test size ratio
boyut= 3 #dimensions

Defining Our Data Set

-----



In [4]:
## Divide data into colors 
def renklere_ayir (data_set):
    colors =[]
    for i in data_set:
        if i==0:
            colors.append('b')
        elif i == 1:
            colors.append('g')
        elif i == 2:
            colors.append('r')
        elif i == 3:
            colors.append('c')
        elif i == 4:
            colors.append('k')
    return colors

Training set:

In [5]:
X_egitim = np.concatenate((np.random.multivariate_normal([2, 2, 2],0.1*np.eye(boyut), buyukluk), 
                        np.random.multivariate_normal([0, -2, -2], 0.1*np.eye(boyut), buyukluk), 
                        np.random.multivariate_normal([-2, 0, 2], 0.1*np.eye(boyut), buyukluk),
                        np.random.multivariate_normal([2, 2, -2], 0.1*np.eye(boyut), buyukluk)))
y_egitim = np.concatenate((np.zeros(buyukluk), np.ones(buyukluk), 2*np.ones(buyukluk), 3*np.ones(buyukluk)))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_egitim[:, 0], X_egitim[:, 1], X_egitim[:, 2], c=renklere_ayir(y_egitim))


Initialize Neurons

In [6]:
x1=10
x2=10
noronlar_boyutu= x1*x2
agirliklar=(np.random.rand(noronlar_boyutu,boyut))
noron_sinifi=4*np.ones(noronlar_boyutu)
ax.scatter(agirliklar[:, 0], agirliklar[:, 1], agirliklar[:, 2], c='k', marker='*') #figure 1
plt.show()

creating the i-j dimension

In [7]:
index_haritasi = np.zeros((noronlar_boyutu, 2))
for i in range(x2):
    for j in range(x1):
        index_haritasi[i*x2 + j, :] = [i, j]
print (index_haritasi)

[[0. 0.]
 [0. 1.]
 [0. 2.]
 [0. 3.]
 [0. 4.]
 [0. 5.]
 [0. 6.]
 [0. 7.]
 [0. 8.]
 [0. 9.]
 [1. 0.]
 [1. 1.]
 [1. 2.]
 [1. 3.]
 [1. 4.]
 [1. 5.]
 [1. 6.]
 [1. 7.]
 [1. 8.]
 [1. 9.]
 [2. 0.]
 [2. 1.]
 [2. 2.]
 [2. 3.]
 [2. 4.]
 [2. 5.]
 [2. 6.]
 [2. 7.]
 [2. 8.]
 [2. 9.]
 [3. 0.]
 [3. 1.]
 [3. 2.]
 [3. 3.]
 [3. 4.]
 [3. 5.]
 [3. 6.]
 [3. 7.]
 [3. 8.]
 [3. 9.]
 [4. 0.]
 [4. 1.]
 [4. 2.]
 [4. 3.]
 [4. 4.]
 [4. 5.]
 [4. 6.]
 [4. 7.]
 [4. 8.]
 [4. 9.]
 [5. 0.]
 [5. 1.]
 [5. 2.]
 [5. 3.]
 [5. 4.]
 [5. 5.]
 [5. 6.]
 [5. 7.]
 [5. 8.]
 [5. 9.]
 [6. 0.]
 [6. 1.]
 [6. 2.]
 [6. 3.]
 [6. 4.]
 [6. 5.]
 [6. 6.]
 [6. 7.]
 [6. 8.]
 [6. 9.]
 [7. 0.]
 [7. 1.]
 [7. 2.]
 [7. 3.]
 [7. 4.]
 [7. 5.]
 [7. 6.]
 [7. 7.]
 [7. 8.]
 [7. 9.]
 [8. 0.]
 [8. 1.]
 [8. 2.]
 [8. 3.]
 [8. 4.]
 [8. 5.]
 [8. 6.]
 [8. 7.]
 [8. 8.]
 [8. 9.]
 [9. 0.]
 [9. 1.]
 [9. 2.]
 [9. 3.]
 [9. 4.]
 [9. 5.]
 [9. 6.]
 [9. 7.]
 [9. 8.]
 [9. 9.]]


defining the dataset

In [9]:
dataset=list(zip(X_egitim,
                 y_egitim))
random.shuffle(dataset)
egitim_kumesi=dataset[:egitim_kumesi_noktalar]
test_kumesi=dataset[egitim_kumesi_noktalar:] #egitim kümesindek inokta sayısından sonrasını alacaktır

COOPERATION

In [10]:
def komsuluk(k, i):
    
    def sigma(k, sigma_0 = 2, sigma_1 = 40):
       return sigma_0 * np.exp(-1 * k / sigma_1)
   
    oranlama = 2 * np.square(sigma(k))
    h = np.zeros(len(index_haritasi))
    for j in range(len(index_haritasi)):
        neuron_dagilimi = np.linalg.norm(index_haritasi[i, :]-index_haritasi[j, :])#np.linalg.norm= matris normunu veriyor
        h[j] = np.exp(-1 * (np.square(neuron_dagilimi)) / oranlama)
    return h

defining the graphs

In [13]:
fig=plt.figure(constrained_layout=True)

#3d figure
ax=fig.add_subplot(projection='3d')
ax.title.set_text ('Veri uzayı')
ax.axis([-5,5,-5,5])
ax.set_zlim([-5,5]) # z axisi
plt.pause(0.3)


test data :


In [16]:
a,b = zip(*test_kumesi)
test_X=np.array(a)
test_y=np.array(b)
ax.scatter(test_X[:,0],test_X[:,1],test_X[:,2],s=10,c=renklere_ayir(test_y))
sc= ax.scatter(agirliklar[:,0],agirliklar[:,1],agirliklar[:,2],c='k',marker='^')

Starting iteration :

In [17]:
iterasyon_sayisi=100
l_rate=0.005
eski_agirliklar=np.zeros(agirliklar.shape) #agirlikları initalize etmişiz gibi sayabiliriz

the algorithm :

In [18]:
try:
    for k in range(iterasyon_sayisi):
        print(k)
        random.shuffle(egitim_kumesi)
        for x,y in egitim_kumesi:
            d=np.zeros(noronlar_boyutu) #dnin initializationu
            for i in range(noronlar_boyutu):
                d[i] = np.linalg.norm(x-agirliklar[i,:]) #np.linalg.norm= matris normunu veriyor #dji nin define edilmesi
            kazanan = np.argmin(d) #minimum norma sahip ola nöron kazanır!
            noron_sinifi[kazanan] = y
            h = komsuluk(k,kazanan)
            for j in range(noronlar_boyutu):
                delta_w = l_rate * h[j] * (x -agirliklar[j]) #unutma terimindeki deta wj
                agirliklar[j] += delta_w # ağırlıkların guncellenmesi
            
        sc.remove()
        sc=ax.scatter(agirliklar[:,0],agirliklar[:,1],agirliklar[:,2],s=10,c=renklere_ayir(noron_sinifi),marker='x')
        plt.pause(0.1)
        if(np.linalg.norm(agirliklar-eski_agirliklar).mean()<0.01): #durma koşulu
          break     
except KeyboardInterrupt:
    pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
